In [116]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from utils.doors import DoorType
from utils.line import Line
from utils.params import Params
plt.style.use('ggplot')
params = Params()
np.random.seed(42) # need to ensure # people generated is same for both line

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [117]:
TotalLineOpen = []
TotalLineNormal = []
for i in range(3):
    TotalLineOpen.append(Line(DoorType.keepOpen,i))
    TotalLineNormal.append(Line(DoorType.normal,i))

In [119]:
for i in range(1000):
    p = [True,False,False]
    for j in range(3):
        TotalLineOpen[j].forward(TotalLineOpen,print = p[j])

😡😡😡😡😡😡😡😡😡😡😡😡😡😡😡😡😡😡😡😡😡😡😡😡😡😡😡😡😡🈳      

KeyboardInterrupt: 